# Calculator Bool Exam

|Entrada| Salida |
|:-:|:-:|
|x := 2;||
|print x;|Resultado es verdadero|
|y := 0 and x; | |
|print nor y;| Resultado es verdadero|
|print x and not y;| Resultado es verdader|
|print not (x and not y) ; | Resultado es falso |
| x:= not x; | |
|z:= true or not (x and not y); ||
|w := (3 > 5)  and (z < 4);||
|print 4 == (w < 3);| Resultado es falso |

